<a href="https://colab.research.google.com/github/Darkcoder1995/Graph-Neural-Network/blob/main/Graphomer_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-geometric

In [ ]:
!pip install torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [ ]:
!pip install sodapy

In [ ]:
!pip install openrouteservice

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.utils import degree, to_networkx
from torch_geometric.data import Data, download_url # Import download_url from data
import networkx as nx
from typing import Tuple, Dict, List
import zipfile # Import the zipfile module
import os # Import os for path joining
from torch_geometric.data import download_url, extract_zip
import pandas as pd

import torch.nn.functional as F
from torch_geometric.loader import DataLoader

import numpy as np
import torch.optim as optim
from torch_geometric.nn import MessagePassing
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.loader import NeighborLoader

In [ ]:
# URL
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

In [ ]:
extract_zip(download_url(url, '.'), '.')

movie_path = 'ml-latest-small/movies.csv'
rating_path = 'ml-latest-small/ratings.csv'
# user_path = 'ml-latest-small/users.csv'

Extracting ./ml-latest-small.zip


In [ ]:
# def floyd_warshall_gpu(adj_matrix):
#     """
#     Computes all-pairs shortest paths using the Floyd-Warshall algorithm on GPU.
#     adj_matrix: Tensor of shape (N, N) with 0 for self-loops and float('inf') for no direct edge.
#     Returns:
#         dist: Tensor of shape (N, N) with shortest path distances.
#     """
#     N = adj_matrix.size(0)
#     dist = adj_matrix.clone().to(torch.float32).to('cuda')
#     for k in range(N):
#         dist = torch.min(dist, dist[:, k].unsqueeze(1) + dist[k, :].unsqueeze(0))
#     return dist

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F

# class CentralityEncoding(nn.Module):
#     def __init__(self, max_in_degree, max_out_degree, node_dim):
#         super().__init__()
#         self.z_in = nn.Parameter(torch.randn(max_in_degree, node_dim))
#         self.z_out = nn.Parameter(torch.randn(max_out_degree, node_dim))

#     def forward(self, x, edge_index):
#         num_nodes = x.size(0)
#         in_deg = torch.bincount(edge_index[1], minlength=num_nodes).clamp(max=self.z_in.size(0) - 1)
#         out_deg = torch.bincount(edge_index[0], minlength=num_nodes).clamp(max=self.z_out.size(0) - 1)
#         return x + self.z_in[in_deg] + self.z_out[out_deg]

# class SpatialEncoding(nn.Module):
#     def __init__(self, max_dist):
#         super().__init__()
#         self.b = nn.Parameter(torch.randn(max_dist))

#     def forward(self, x, dist_matrix):
#         dist_matrix = dist_matrix.clamp(max=self.b.size(0) - 1).long()
#         return self.b[dist_matrix]

# class GraphormerAttentionHead(nn.Module):
#     def __init__(self, dim_in, dim_qk, dim_v):
#         super().__init__()
#         self.q = nn.Linear(dim_in, dim_qk)
#         self.k = nn.Linear(dim_in, dim_qk)
#         self.v = nn.Linear(dim_in, dim_v)

#     def forward(self, x, bias):
#         q = self.q(x)
#         k = self.k(x)
#         v = self.v(x)
#         attn_scores = (q @ k.transpose(0, 1)) / q.size(-1) ** 0.5
#         attn_scores = attn_scores + bias
#         attn_weights = F.softmax(attn_scores, dim=-1)
#         return attn_weights @ v

# class GraphormerRecommender(nn.Module):
#     def __init__(self, node_dim, max_in_degree, max_out_degree, max_path_distance):
#         super().__init__()
#         self.centrality = CentralityEncoding(max_in_degree, max_out_degree, node_dim)
#         self.spatial = SpatialEncoding(max_path_distance)
#         self.attn = GraphormerAttentionHead(node_dim, node_dim, node_dim)
#         self.out = nn.Linear(node_dim, 1)

#     def forward(self, x, edge_index, dist_matrix):
#         x = self.centrality(x, edge_index)
#         bias = self.spatial(x, dist_matrix)
#         x = self.attn(x, bias)
#         return self.out(x).squeeze(-1)

In [ ]:
# ratings = pd.read_csv(rating_path)
# movies = pd.read_csv(movie_path)

In [ ]:
# user_id_map = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
# movie_id_map = {mid: i + len(user_id_map) for i, mid in enumerate(ratings['movieId'].unique())}

In [ ]:
# movies

In [ ]:
# def load_movielens_data(movie_path, rating_path, min_ratings_per_user=10):
#     ratings = pd.read_csv(rating_path)
#     movies = pd.read_csv(movie_path)

#     # Filter to active users only
#     user_counts = ratings['userId'].value_counts()
#     active_users = user_counts[user_counts >= min_ratings_per_user].index
#     ratings = ratings[ratings['userId'].isin(active_users)]

#     user_id_map = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#     movie_id_map = {mid: i + len(user_id_map) for i, mid in enumerate(ratings['movieId'].unique())}

#     edge_index = []
#     edge_attr = []
#     for _, row in ratings.iterrows():
#         u, m, r = row['userId'], row['movieId'], row['rating']
#         uid = user_id_map[u]
#         mid = movie_id_map[m]
#         edge_index.append([uid, mid])
#         edge_index.append([mid, uid])
#         edge_attr.append([r])
#         edge_attr.append([r])

#     edge_index = torch.tensor(edge_index).t().contiguous()
#     edge_attr = torch.tensor(edge_attr, dtype=torch.float)

#     num_nodes = len(user_id_map) + len(movie_id_map)
#     x = torch.randn((num_nodes, 64))

#     # Create adjacency matrix
#     adj_matrix = torch.full((num_nodes, num_nodes), float('inf'))
#     adj_matrix[torch.arange(num_nodes), torch.arange(num_nodes)] = 0
#     for i in range(edge_index.size(1)):
#         src = edge_index[0, i]
#         dst = edge_index[1, i]
#         adj_matrix[src, dst] = 1  # Assuming unweighted edges for shortest path

#     # Compute shortest path distances
#     dist_matrix = floyd_warshall_gpu(adj_matrix)

#     data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
#     data.dist_matrix = dist_matrix
#     data.user_id_map = user_id_map
#     data.movie_id_map = movie_id_map
#     return data

In [ ]:
# def train(model, data, epochs=50, lr=1e-3, batch_size=64):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = model.to(device)
#     data = data.to(device)

#     # Optimizer & Learning Rate Scheduler
#     optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

#     # Use NeighborLoader for mini-batch training
#     train_loader = NeighborLoader(
#         data, num_neighbors=[10] * 2, batch_size=batch_size, input_nodes=data.train_mask
#     )

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         for batch in train_loader:
#             batch = batch.to(device)
#             optimizer.zero_grad()

#             # Forward pass (include edge attributes!)
#             out = model(batch.x, batch.edge_index, batch.edge_attr)
#             preds = out[batch.edge_index[1]]
#             targets = batch.edge_attr.squeeze()

#             # Loss & Backpropagation
#             loss = F.mse_loss(preds, targets)
#             loss.backward()

#             # Apply gradient clipping
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

#             optimizer.step()
#             total_loss += loss.item() * batch.num_nodes

#         total_loss /= len(train_loader.dataset)
#         scheduler.step(total_loss)  # Adjust LR based on loss trend

#         print(f"Epoch {epoch + 1}/{epochs} | Loss: {total_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

In [ ]:
# # Paths to the MovieLens dataset
# movie_path = 'ml-latest-small/movies.csv'
# rating_path = 'ml-latest-small/ratings.csv'

# # Load data
# data = load_movielens_data(movie_path, rating_path)

# # Create model
# model = GraphormerRecommender(
#     node_dim=64,
#     max_in_degree=10,
#     max_out_degree=10,
#     max_path_distance=10
# )

# # Train model
# train(model, data)

In [ ]:
# # --- Floyd-Warshall GPU ---
# def floyd_warshall_gpu(adj_matrix):
#     N = adj_matrix.size(0)
#     dist = adj_matrix.clone().float().to('cuda')
#     for k in range(N):
#         dist = torch.min(dist, dist[:, k].unsqueeze(1) + dist[k, :].unsqueeze(0))
#     return dist

In [ ]:
# # --- Centrality Encoding ---
# class CentralityEncoding(nn.Module):
#     def __init__(self, max_in_deg, max_out_deg, node_dim):
#         super().__init__()
#         self.z_in = nn.Parameter(torch.randn(max_in_deg, node_dim))
#         self.z_out = nn.Parameter(torch.randn(max_out_deg, node_dim))

#     def forward(self, x, edge_index):
#         num_nodes = x.size(0)
#         in_deg = torch.bincount(edge_index[1], minlength=num_nodes).clamp(max=self.z_in.size(0) - 1)
#         out_deg = torch.bincount(edge_index[0], minlength=num_nodes).clamp(max=self.z_out.size(0) - 1)
#         return x + self.z_in[in_deg] + self.z_out[out_deg]

In [ ]:
# # --- Spatial Encoding ---
# class SpatialEncoding(nn.Module):
#     def __init__(self, max_dist, node_dim):
#         super().__init__()
#         self.b = nn.Parameter(torch.randn(max_dist, node_dim))

#     def forward(self, dist_matrix):
#         dist_matrix = dist_matrix.clamp(max=self.b.size(0) - 1).long()
#         return self.b[dist_matrix]

In [ ]:
# # --- Attention ---
# class GraphormerAttentionHead(nn.Module):
#     def __init__(self, dim_in):
#         super().__init__()
#         self.q = nn.Linear(dim_in, dim_in)
#         self.k = nn.Linear(dim_in, dim_in)
#         self.v = nn.Linear(dim_in, dim_in)

#     def forward(self, x, bias):
#         q, k, v = self.q(x), self.k(x), self.v(x)
#         attn = (q @ k.T) / (q.size(-1) ** 0.5)
#         attn += bias.sum(dim=-1)  # [N, N]
#         attn = F.softmax(attn, dim=-1)
#         return attn @ v

In [ ]:
# # --- Graphormer Model ---
# class GraphormerRecommender(nn.Module):
#     def __init__(self, node_dim, max_in_deg, max_out_deg, max_dist):
#         super().__init__()
#         self.centrality = CentralityEncoding(max_in_deg, max_out_deg, node_dim)
#         self.spatial = SpatialEncoding(max_dist, node_dim)
#         self.attn = GraphormerAttentionHead(node_dim)
#         self.out = nn.Linear(node_dim, 1)

#     def forward(self, x, edge_index, dist_matrix):
#         x = self.centrality(x, edge_index)
#         bias = self.spatial(dist_matrix)
#         x = self.attn(x, bias)
#         return self.out(x).squeeze(-1)

In [ ]:
# # --- Load Data ---
# def load_movielens(movie_path, rating_path, min_ratings=10):
#     ratings = pd.read_csv(rating_path)
#     movies = pd.read_csv(movie_path)
#     users = ratings['userId'].value_counts()
#     active_users = users[users >= min_ratings].index
#     ratings = ratings[ratings['userId'].isin(active_users)]

#     uid_map = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#     mid_map = {mid: i + len(uid_map) for i, mid in enumerate(ratings['movieId'].unique())}

#     edge_index, edge_attr = [], []
#     for _, row in ratings.iterrows():
#         uid = uid_map[row['userId']]
#         mid = mid_map[row['movieId']]
#         edge_index += [[uid, mid], [mid, uid]]
#         edge_attr += [[row['rating']], [row['rating']]]

#     edge_index = torch.tensor(edge_index).T.contiguous()
#     edge_attr = torch.tensor(edge_attr, dtype=torch.float)
#     num_nodes = len(uid_map) + len(mid_map)
#     x = torch.randn((num_nodes, 64))

#     adj = torch.full((num_nodes, num_nodes), float('inf'))
#     adj[torch.arange(num_nodes), torch.arange(num_nodes)] = 0
#     for src, dst in edge_index.T:
#         adj[src, dst] = 1

#     dist_matrix = floyd_warshall_gpu(adj)
#     data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
#     data.dist_matrix = dist_matrix
#     return data

In [ ]:
# # --- Training ---
# def train(model, data, epochs=10):
#     model = model.to('cuda')
#     data = data.to('cuda')
#     opt = torch.optim.Adam(model.parameters(), lr=0.001)
#     loss_fn = nn.MSELoss()

#     for epoch in range(epochs):
#         model.train()
#         opt.zero_grad()
#         out = model(data.x, data.edge_index, data.dist_matrix)
#         pred = out[data.edge_index[0]]
#         target = data.edge_attr.squeeze()
#         loss = loss_fn(pred, target)
#         loss.backward()
#         opt.step()
#         print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
# # --- Run ---
# movie_path = 'ml-latest-small/movies.csv'
# rating_path = 'ml-latest-small/ratings.csv'

# data = load_movielens(movie_path, rating_path)
# model = GraphormerRecommender(node_dim=64, max_in_deg=10, max_out_deg=10, max_dist=10)
# train(model, data, epochs=10)

In [ ]:
# ---- Graphormer Components ---- #

class CentralityEncoding(nn.Module):
    def __init__(self, max_in_degree, max_out_degree, node_dim):
        super().__init__()
        self.z_in = nn.Parameter(torch.randn(max_in_degree, node_dim))
        self.z_out = nn.Parameter(torch.randn(max_out_degree, node_dim))

    def forward(self, x, edge_index):
        num_nodes = x.size(0)
        in_deg = torch.bincount(edge_index[1], minlength=num_nodes).clamp(max=self.z_in.size(0) - 1)
        out_deg = torch.bincount(edge_index[0], minlength=num_nodes).clamp(max=self.z_out.size(0) - 1)
        return x + self.z_in[in_deg] + self.z_out[out_deg]

In [ ]:
class SpatialBias(nn.Module):
    def __init__(self, max_hops, dim):
        super().__init__()
        self.bias = nn.Embedding(max_hops + 1, dim)

    def forward(self, edge_index, num_nodes):
        edge_hops = torch.ones(edge_index.size(1), dtype=torch.long, device=edge_index.device)
        edge_hops = edge_hops.clamp(max=self.bias.num_embeddings - 1)
        return self.bias(edge_hops)  # [E, dim]

In [ ]:
class GraphormerLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.q = nn.Linear(dim, dim)
        self.k = nn.Linear(dim, dim)
        self.v = nn.Linear(dim, dim)
        self.proj = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x, edge_index, spatial_bias):
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        row, col = edge_index
        alpha = (q[row] * k[col]).sum(dim=-1) / q.size(-1) ** 0.5
        alpha = alpha + spatial_bias.sum(dim=-1)
        alpha = torch.softmax(alpha, dim=0)
        out = torch.zeros_like(x)
        out.index_add_(0, row, alpha.unsqueeze(-1) * v[col])
        return self.dropout(self.proj(out))

In [ ]:
class GraphTransformerNet(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_layers, max_hops=1):
        super().__init__()
        self.input_proj = nn.Linear(in_dim, hidden_dim)
        self.centrality = CentralityEncoding(32, 32, hidden_dim)
        self.spatial_bias = SpatialBias(max_hops, hidden_dim)
        self.layers = nn.ModuleList([GraphormerLayer(hidden_dim) for _ in range(num_layers)])
        self.output_layer = nn.Linear(hidden_dim, 1)
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.input_proj(x)
        x = self.centrality(x, edge_index)
        spatial_bias = self.spatial_bias(edge_index, x.size(0))
        for layer in self.layers:
            x = layer(x, edge_index, spatial_bias)
        x = self.norm(x)
        return self.output_layer(x).squeeze(-1)

In [ ]:
# ---- Dataset Loader ---- #

def load_movielens_data(movie_path, rating_path, min_ratings_per_user=10):
    ratings = pd.read_csv(rating_path)
    movies = pd.read_csv(movie_path)

    user_counts = ratings['userId'].value_counts()
    active_users = user_counts[user_counts >= min_ratings_per_user].index
    ratings = ratings[ratings['userId'].isin(active_users)]

    user_id_map = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
    movie_id_map = {mid: i + len(user_id_map) for i, mid in enumerate(ratings['movieId'].unique())}

    edge_index = []
    edge_attr = []
    y = torch.zeros(len(user_id_map) + len(movie_id_map))

    for _, row in ratings.iterrows():
        uid = user_id_map[row['userId']]
        mid = movie_id_map[row['movieId']]
        rating = row['rating']
        edge_index.append([uid, mid])
        edge_index.append([mid, uid])
        edge_attr.append([rating])
        edge_attr.append([rating])
        y[mid] = rating  # Supervision on item nodes (e.g., movies)

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    num_nodes = len(user_id_map) + len(movie_id_map)
    x = torch.randn((num_nodes, 64))  # Random init for node features

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    movie_ids = list(movie_id_map.values())
    split = int(0.8 * len(movie_ids))
    train_ids = movie_ids[:split]
    test_ids = movie_ids[split:]
    data.train_mask[train_ids] = True
    data.test_mask[test_ids] = True

    return data

In [ ]:
# ---- Main Execution ---- #

if __name__ == '__main__':
    movie_path = 'ml-latest-small/movies.csv'
    rating_path = 'ml-latest-small/ratings.csv'

    data = load_movielens_data(movie_path, rating_path)

    train_loader = NeighborLoader(data, input_nodes=data.train_mask, num_neighbors=[10, 10], batch_size=64)
    val_loader = NeighborLoader(data, input_nodes=data.test_mask, num_neighbors=[10, 10], batch_size=64)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GraphTransformerNet(in_dim=data.x.size(1), hidden_dim=128, num_layers=3).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
    loss_fn = nn.MSELoss()

    for epoch in range(1, 51):
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            pred = model(batch)
            loss = loss_fn(pred[batch.train_mask], batch.y[batch.train_mask])
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * batch.train_mask.sum().item()

        print(f"Epoch {epoch:03d}, Train Loss: {total_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


Epoch 001, Train Loss: 189489.6700
Epoch 002, Train Loss: 106224.2420
Epoch 003, Train Loss: 105431.3456
Epoch 004, Train Loss: 105477.7980
Epoch 005, Train Loss: 105860.1963
Epoch 006, Train Loss: 105184.0393
Epoch 007, Train Loss: 106582.7647
Epoch 008, Train Loss: 105877.6009
Epoch 009, Train Loss: 105858.9067
Epoch 010, Train Loss: 105602.5273
Epoch 011, Train Loss: 104621.6932
Epoch 012, Train Loss: 105142.2777
Epoch 013, Train Loss: 105325.0419
Epoch 014, Train Loss: 105198.4142
Epoch 015, Train Loss: 104121.1250
Epoch 016, Train Loss: 104699.6626
Epoch 017, Train Loss: 104276.4174
Epoch 018, Train Loss: 103794.4357
Epoch 019, Train Loss: 103328.7948
Epoch 020, Train Loss: 104387.4779
Epoch 021, Train Loss: 103912.6172
Epoch 022, Train Loss: 105020.6069
Epoch 023, Train Loss: 104283.7223
Epoch 024, Train Loss: 104444.5142
Epoch 025, Train Loss: 104339.4304
Epoch 026, Train Loss: 103493.9160
Epoch 027, Train Loss: 104436.8517
Epoch 028, Train Loss: 103381.8570
Epoch 029, Train Los

In [ ]:
# ---- Evaluation on Test Data ----
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        out = model(batch)
        preds = out[batch.test_mask].cpu().numpy()
        targets = batch.y[batch.test_mask].cpu().numpy()

        y_pred.extend(preds)
        y_true.extend(targets)

# Compute metrics
mse = mean_squared_error(y_true, y_pred)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

print(f"\n📊 Test MSE: {mse:.4f}")
print(f"📉 Test RMSE: {rmse:.4f}")
print(f"🎯 Test MAE: {mae:.4f}")


📊 Test MSE: 1.3128
📉 Test RMSE: 1.1458
🎯 Test MAE: 0.8881


In [ ]:
# 📊 Test MSE: 1.3128
# 📉 Test RMSE: 1.1458
# 🎯 Test MAE: 0.8881

In [ ]:
# def train(model, data, epochs=30, lr=1e-3):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = model.to(device)
#     data = data.to(device)
#     optimizer = optim.Adam(model.parameters(), lr=lr)
#     model.train()

#     user_ids = list(data.user_id_map.values())
#     movie_ids = list(data.movie_id_map.values())
#     user_mask = torch.tensor([i in user_ids for i in data.edge_index[0]], device=device)
#     movie_mask = torch.tensor([i in movie_ids for i in data.edge_index[1]], device=device)
#     mask = user_mask & movie_mask

#     for epoch in range(epochs):
#         optimizer.zero_grad()
#         out = model(data.x, data.edge_index, data.dist_matrix)
#         preds = out[data.edge_index[1][mask]]
#         targets = data.edge_attr.squeeze()[mask]
#         loss = F.mse_loss(preds, targets)
#         loss.backward()
#         optimizer.step()
#         print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss.item():.4f}")

In [ ]:
# # === utils.py ===
# def floyd_warshall_source_to_all(G, source, cutoff=None):
#     if source not in G:
#         raise nx.NodeNotFound(f"Source {source} not in G")

#     edges = {edge: i for i, edge in enumerate(G.edges())}
#     level = 0
#     nextlevel = {source: 1}
#     node_paths = {source: [source]}
#     edge_paths = {source: []}

#     while nextlevel:
#         thislevel = nextlevel
#         nextlevel = {}
#         for v in thislevel:
#             for w in G[v]:
#                 if w not in node_paths:
#                     node_paths[w] = node_paths[v] + [w]
#                     edge_paths[w] = edge_paths[v] + [edges.get((node_paths[w][-2], w), 0)]
#                     nextlevel[w] = 1
#         level += 1
#         if cutoff is not None and level >= cutoff:
#             break

#     return node_paths, edge_paths

# def all_pairs_shortest_path(G) -> Tuple[Dict[int, Dict[int, List[int]]], Dict[int, Dict[int, List[int]]]]:
#     node_paths, edge_paths = {}, {}
#     for node in G:
#         n_path, e_path = floyd_warshall_source_to_all(G, node)
#         node_paths[node] = n_path
#         edge_paths[node] = e_path
#     return node_paths, edge_paths

# def shortest_path_distance(data: Data):
#     G = to_networkx(data)
#     return all_pairs_shortest_path(G)


# # === layers.py ===
# class CentralityEncoding(nn.Module):
#     def __init__(self, max_in_degree, max_out_degree, node_dim):
#         super().__init__()
#         self.z_in = nn.Parameter(torch.randn(max_in_degree, node_dim))
#         self.z_out = nn.Parameter(torch.randn(max_out_degree, node_dim))

#     def forward(self, x, edge_index):
#         num_nodes = x.size(0)
#         in_deg = degree(edge_index[1], num_nodes=num_nodes).long().clamp(max=self.z_in.size(0) - 1)
#         out_deg = degree(edge_index[0], num_nodes=num_nodes).long().clamp(max=self.z_out.size(0) - 1)
#         return x + self.z_in[in_deg] + self.z_out[out_deg]

# class SpatialEncoding(nn.Module):
#     def __init__(self, max_dist):
#         super().__init__()
#         self.b = nn.Parameter(torch.randn(max_dist))

#     def forward(self, x, paths):
#         mat = torch.zeros((x.size(0), x.size(0)), device=x.device)
#         for i in paths:
#             for j in paths[i]:
#                 d = min(len(paths[i][j]), len(self.b)) - 1
#                 mat[i][j] = self.b[d]
#         return mat

# def dot_product(x1, x2):
#     return (x1 * x2).sum(dim=1)

# class EdgeEncoding(nn.Module):
#     def __init__(self, edge_dim, max_dist):
#         super().__init__()
#         self.edge_vector = nn.Parameter(torch.randn(max_dist, edge_dim))

#     def forward(self, x, edge_attr, edge_paths):
#         size = x.size(0)
#         cij = torch.zeros((size, size), device=x.device)
#         for i in edge_paths:
#             for j in edge_paths[i]:
#                 path = edge_paths[i][j][:len(self.edge_vector)]
#                 idx = torch.tensor(path, dtype=torch.long, device=x.device)
#                 if len(idx) == 0:
#                     continue
#                 cij[i][j] = dot_product(self.edge_vector[:len(idx)], edge_attr[idx]).mean()
#         return torch.nan_to_num(cij)

# class GraphormerAttentionHead(nn.Module):
#     def __init__(self, dim_in, dim_q, dim_k, edge_dim, max_dist):
#         super().__init__()
#         self.edge_enc = EdgeEncoding(edge_dim, max_dist)
#         self.q = nn.Linear(dim_in, dim_q)
#         self.k = nn.Linear(dim_in, dim_k)
#         self.v = nn.Linear(dim_in, dim_k)

#     def forward(self, x, edge_attr, b, edge_paths, ptr=None):
#         device = x.device
#         mask_inf = torch.full((x.size(0), x.size(0)), -1e6, device=device)
#         mask_zero = torch.zeros_like(mask_inf)

#         if ptr is None:
#             mask_inf.fill_(1)
#             mask_zero.fill_(1)
#         else:
#             for i in range(len(ptr) - 1):
#                 mask_inf[ptr[i]:ptr[i+1], ptr[i]:ptr[i+1]] = 1
#                 mask_zero[ptr[i]:ptr[i+1], ptr[i]:ptr[i+1]] = 1

#         q, k, v = self.q(x), self.k(x), self.v(x)
#         c = self.edge_enc(x, edge_attr, edge_paths)
#         a = self.compute_a(q, k, ptr)
#         a = (a + b + c) * mask_inf
#         alpha = torch.softmax(a, dim=-1) * mask_zero
#         return alpha @ v

#     def compute_a(self, q, k, ptr=None):
#         if ptr is None:
#             return q @ k.t() / q.size(-1) ** 0.5
#         a = torch.zeros(q.size(0), q.size(0), device=q.device)
#         for i in range(len(ptr) - 1):
#             a[ptr[i]:ptr[i+1], ptr[i]:ptr[i+1]] = (
#                 q[ptr[i]:ptr[i+1]] @ k[ptr[i]:ptr[i+1]].t()
#             ) / q.size(-1) ** 0.5
#         return a


# # === Model Integration ===
# class GraphormerRecommender(nn.Module):
#     def __init__(self, node_dim, edge_dim, max_in_degree, max_out_degree, max_path_distance):
#         super().__init__()
#         self.centrality = CentralityEncoding(max_in_degree, max_out_degree, node_dim)
#         self.spatial = SpatialEncoding(max_path_distance)
#         self.attn = GraphormerAttentionHead(node_dim, node_dim, node_dim, edge_dim, max_path_distance)
#         self.out = nn.Linear(node_dim, 1)  # Predict rating or score

#     def forward(self, data):
#         x, edge_index, edge_attr, ptr = data.x, data.edge_index, data.edge_attr, data.ptr
#         edge_paths = data.edge_paths
#         node_paths = data.node_paths

#         x = self.centrality(x, edge_index)
#         b = self.spatial(x, node_paths)
#         x = self.attn(x, edge_attr, b, edge_paths, ptr)
#         return self.out(x).squeeze(-1)  # Final output as score per node


# # This architecture can now be trained with a dataset like MovieLens,
# # where users and items are nodes, and interactions are edges with ratings.
# # Ensure `data` passed to `forward` contains `x`, `edge_index`, `edge_attr`, `ptr`, `edge_paths`, `node_paths`.


In [ ]:
# def approximate_shortest_path_distance(edge_index, num_nodes, max_hops=5):
#     G = nx.Graph()
#     G.add_edges_from(edge_index.t().tolist())

#     # Limit to k-hop paths
#     node_paths = torch.full((num_nodes, num_nodes), float('inf'))
#     for node in range(num_nodes):
#         sp = nx.single_source_shortest_path_length(G, node, cutoff=max_hops)
#         for target, dist in sp.items():
#             node_paths[node, target] = dist

#     node_paths[node_paths == float('inf')] = max_hops + 1
#     return node_paths

In [ ]:
# def floyd_warshall_source_to_all(G, source, cutoff=None):
#     if source not in G:
#         raise nx.NodeNotFound(f"Source {source} not in G")

#     edges = {edge: i for i, edge in enumerate(G.edges())}
#     level = 0
#     nextlevel = {source: 1}
#     node_paths = {source: [source]}
#     edge_paths = {source: []}

#     while nextlevel:
#         thislevel = nextlevel
#         nextlevel = {}
#         for v in thislevel:
#             for w in G[v]:
#                 if w not in node_paths:
#                     node_paths[w] = node_paths[v] + [w]
#                     edge_paths[w] = edge_paths[v] + [edges.get((node_paths[w][-2], w), 0)]
#                     nextlevel[w] = 1
#         level += 1
#         if cutoff is not None and level >= cutoff:
#             break

#     return node_paths, edge_paths

In [ ]:
# def load_movielens_data(movie_path, rating_path, min_ratings_per_user=10):
#     ratings = pd.read_csv(rating_path)
#     movies = pd.read_csv(movie_path)

#     # Filter to active users only
#     user_counts = ratings['userId'].value_counts()
#     active_users = user_counts[user_counts >= min_ratings_per_user].index
#     ratings = ratings[ratings['userId'].isin(active_users)]

#     user_id_map = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#     movie_id_map = {mid: i + len(user_id_map) for i, mid in enumerate(ratings['movieId'].unique())}

#     edge_index, edge_attr, x = [], [], []
#     for _, row in ratings.iterrows():
#         u, m, r = row['userId'], row['movieId'], row['rating']
#         uid = user_id_map[u]
#         mid = movie_id_map[m]

#         edge_index.append([uid, mid])
#         edge_index.append([mid, uid])
#         edge_attr.append([r])
#         edge_attr.append([r])

#     edge_index = torch.tensor(edge_index).t().contiguous()
#     edge_attr = torch.tensor(edge_attr, dtype=torch.float)

#     num_nodes = len(user_id_map) + len(movie_id_map)
#     x = torch.randn((num_nodes, 64))  # Node features

#     data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

#     # Convert to networkx and compute shortest paths
#     G = nx.Graph()
#     G.add_edges_from(edge_index.t().tolist())
#     node_paths, edge_paths = shortest_path_distance(data)

#     data.node_paths = node_paths
#     data.edge_paths = edge_paths
#     data.ptr = torch.tensor([0, num_nodes])  # Single graph

#     return data

In [ ]:
# def train(model, data, epochs=30, lr=1e-3):
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     model.train()

#     for epoch in range(epochs):
#         optimizer.zero_grad()

#         out = model(data)

#         # Use only user -> movie edges for supervision
#         mask = data.edge_index[0] < len(set(data.edge_index[0].tolist()))
#         targets = data.edge_attr.squeeze()[mask]
#         preds = out[data.edge_index[1][mask]]

#         loss = F.mse_loss(preds, targets)
#         loss.backward()
#         optimizer.step()

#         print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss.item():.4f}")

In [ ]:
# # Paths
# movie_path = 'ml-latest-small/movies.csv'
# rating_path = 'ml-latest-small/ratings.csv'

# # Load data
# data = load_movielens_data(movie_path, rating_path)

# # Create model
# model = GraphormerRecommender(
#     node_dim=64,
#     edge_dim=1,
#     max_in_degree=10,
#     max_out_degree=10,
#     max_path_distance=10
# )

# # Train model
# train(model, data)